If the cell raises error of  "No module named 'chocoq'", please open a new terminal and excute
```bash
conda activate chocoq_cpu
```
or 
```bash
conda activate chocoq_gpu
```
enter into the `Choco-Q/` dir and excute
```bash
pip install .
```
reopen this notebook and rerun the cells.

## import the packages

In [1]:
should_print = True # True for debug model and False for release mode.

import os
os.chdir("../..")
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARN)

from janusq.application.chocoq.chocoq.model import LinearConstrainedBinaryOptimization as LcboModel
from janusq.application.chocoq.chocoq.solvers.optimizers import CobylaOptimizer, AdamOptimizer
from janusq.application.chocoq.chocoq.solvers.qiskit import (
    PenaltySolver, CyclicSolver, HeaSolver, ChocoSolver, 
    AerGpuProvider, AerProvider, FakeBrisbaneProvider, FakeKyivProvider, FakeTorinoProvider, DdsimProvider,
)

## customize the constrained problem

In [ ]:
# ----------------------- model -----------------------
m = LcboModel()
x = m.addVars(5, name="x") ## add five binary variables with prefix name x
m.setObjective((x[0] + x[1])* x[3] + x[2], "max") ## set the maximum objective with (x[0]+x[1])x[3] + x[2]
# m.addConstr(x[0] + x[1] + x[2] == 2)
# m.addConstr(x[0] + x[1] == 1)

m.addConstr(x[0] + x[1] - x[2] == 0) #add linear constraints
m.addConstr(x[2] + x[3] - x[4] == 1)

print(m.lin_constr_mtx) #print linear constraints matrix
print(m) # print the information of model.

The exact solution is

In [ ]:
optimize = m.optimize()
print(f"optimize_cost: {optimize}\n\n")

Solve the optimization problem by Choco-Q.

In [ ]:
# sovler ----------------------------------------------
opt = CobylaOptimizer(max_iter=200) # set the optimizer COBYLA, the max iteration is 200.
# gpu = AerGpuProvider()
aer = DdsimProvider()
solver = ChocoSolver(
    prb_model=m,  # the problem model
    optimizer=opt,  # the optimizer for parameters updating.
    provider=aer,  # quantum circuit simulator.
    num_layers=1,
)
print(solver.circuit_analyze(['depth', 'width', 'culled_depth', 'num_one_qubit_gates'])) 
## the circuit analyze results

In [ ]:
result = solver.solve() # solving with input solver
print(result)

In [ ]:
eval = solver.evaluation() # evaluation of the solving results
print(eval) 